In [27]:
# Import all the required libraries
import fastf1
import pandas as pd     

# Enable caching for faster data loading
fastf1.Cache.enable_cache('cache')

In [28]:
# Import te results df
results_df = pd.read_csv('DATA/f1_results_2024_2025.csv')
results_df.describe

<bound method NDFrame.describe of      DriverNumber BroadcastName Abbreviation         DriverId  \
0              11       S PEREZ          PER            perez   
1              24        G ZHOU          ZHO             zhou   
2              20   K MAGNUSSEN          MAG  kevin_magnussen   
3               3   D RICCIARDO          RIC        ricciardo   
4              77      V BOTTAS          BOT           bottas   
..            ...           ...          ...              ...   
170            22     Y TSUNODA          TSU          tsunoda   
171            14      F ALONSO          ALO           alonso   
172            16     C LECLERC          LEC          leclerc   
173            44    L HAMILTON          HAM         hamilton   
174            10       P GASLY          GAS            gasly   

            TeamName TeamColor        TeamId FirstName   LastName  \
0    Red Bull Racing    3671c6      red_bull    Sergio      Perez   
1        Kick Sauber    52e252        sauber   

In [29]:
%pip install openpyxl

# Load driver data from an Excel file
drivers = pd.read_excel('DATA/f1_drivers_points_exp.xlsx')
drivers

Note: you may need to restart the kernel to use updated packages.


,DriverName,Abbreviation,TeamId,DriverExperience
0,Max Verstappen,VER,red_bull,11
1,Lewis Hamilton,HAM,ferrari,19
2,Charles Leclerc,LEC,ferrari,8
3,Carlos Sainz,SAI,williams,11
4,Lando Norris,NOR,mclaren,7
5,George Russell,RUS,mercedes,7
6,Fernando Alonso,ALO,aston_martin,22
7,Oscar Piastri,PIA,mclaren,3
8,Esteban Ocon,OCO,haas,9
9,Pierre Gasly,GAS,alpine,9


In [30]:
filtered_results = results_df[results_df['Abbreviation'].isin(drivers['Abbreviation'])]

# Group by driver and sum total points
total_points = filtered_results.groupby('Abbreviation')['Points'].sum().reset_index()

# Fill the missing rookie drivers with 0 points
all_drivers_df = pd.DataFrame({'Abbreviation': drivers['Abbreviation']})
total_points = all_drivers_df.merge(total_points, on = 'Abbreviation', how = 'left').fillna(0)

# Convert points to absolute value integer for simplerer point system
total_points["Points"] = total_points["Points"].astype(int)
total_points = total_points.sort_values(by="Points", ascending=False)

total_points

,Abbreviation,Points
4,NOR,43
0,VER,30
5,RUS,30
7,PIA,27
17,ANT,20
12,ALB,16
8,OCO,10
10,STR,10
13,HUL,6
14,BEA,4


In [31]:
drivers = total_points.merge(drivers, on = 'Abbreviation', how = 'left').fillna(0)
drivers

,Abbreviation,Points,DriverName,TeamId,DriverExperience
0,NOR,43,Lando Norris,mclaren,7
1,VER,30,Max Verstappen,red_bull,11
2,RUS,30,George Russell,mercedes,7
3,PIA,27,Oscar Piastri,mclaren,3
4,ANT,20,Kimi Antonelli,mercedes,1
5,ALB,16,Alexander Albon,williams,6
6,OCO,10,Esteban Ocon,haas,9
7,STR,10,Lance Stroll,aston_martin,9
8,HUL,6,Nico Hülkenberg,sauber,13
9,BEA,4,Oliver Bearman,haas,1


In [32]:
# Calculate average qualifying position for each driver
avg_quali_df = results_df.groupby('Abbreviation')['GridPosition'].mean().reset_index()
avg_quali_df.rename(columns = {'GridPosition': 'AvgQualiPosition'}, inplace = True)


In [33]:
# Calculate average race position for each driver
avg_race_df = results_df.groupby('Abbreviation')['Position'].mean().reset_index()
avg_race_df.rename(columns={'Position': 'AvgRacePosition'}, inplace=True)

In [34]:
# Merge both dataframes
avg_pos_df = avg_quali_df.merge(avg_race_df, on = 'Abbreviation', how = 'outer')

# Merge the drivers DataFrame with avg_pos_df to include AvgQualiPosition and AvgRacePosition
drivers = drivers.merge(avg_pos_df, on = 'Abbreviation', how = 'left')

drivers

,Abbreviation,Points,DriverName,TeamId,DriverExperience,AvgQualiPosition,AvgRacePosition
0,NOR,43,Lando Norris,mclaren,7,2.0,1.5
1,VER,30,Max Verstappen,red_bull,11,3.5,3.0
2,RUS,30,George Russell,mercedes,7,3.0,3.0
3,PIA,27,Oscar Piastri,mclaren,3,1.5,5.0
4,ANT,20,Kimi Antonelli,mercedes,1,12.0,5.0
5,ALB,16,Alexander Albon,williams,6,8.0,6.0
6,OCO,10,Esteban Ocon,haas,9,15.0,9.0
7,STR,10,Lance Stroll,aston_martin,9,13.5,7.5
8,HUL,6,Nico Hülkenberg,sauber,13,14.5,11.0
9,BEA,4,Oliver Bearman,haas,1,18.5,11.0


In [38]:
filtered_drivers_info = drivers.drop(['DriverExperience'], axis = 1)
filtered_drivers_info

,Abbreviation,Points,DriverName,TeamId,AvgQualiPosition,AvgRacePosition
0,NOR,43,Lando Norris,mclaren,2.0,1.5
1,VER,30,Max Verstappen,red_bull,3.5,3.0
2,RUS,30,George Russell,mercedes,3.0,3.0
3,PIA,27,Oscar Piastri,mclaren,1.5,5.0
4,ANT,20,Kimi Antonelli,mercedes,12.0,5.0
5,ALB,16,Alexander Albon,williams,8.0,6.0
6,OCO,10,Esteban Ocon,haas,15.0,9.0
7,STR,10,Lance Stroll,aston_martin,13.5,7.5
8,HUL,6,Nico Hülkenberg,sauber,14.5,11.0
9,BEA,4,Oliver Bearman,haas,18.5,11.0


In [36]:
filtered_drivers_info.to_csv('DATA/filtred_drivers_info.csv')

In [40]:
# Select only relevant features from results_df for further processing
filtered_results_df = results_df[['Round', 'Abbreviation', 'GridPosition', 'Position']]
filtered_results_df

,Round,Abbreviation,GridPosition,Position
0,1,PER,5.0,2.0
1,1,ZHO,17.0,11.0
2,1,MAG,15.0,12.0
3,1,RIC,14.0,13.0
4,1,BOT,16.0,19.0
...,...,...,...,...
170,2,TSU,9.0,16.0
171,2,ALO,13.0,17.0
172,2,LEC,6.0,18.0
173,2,HAM,5.0,19.0


In [49]:
# Merge the filtered race results with driver information using 'Abbreviation' as the key
# This combines race performance with driver statistics and gives the final dataset to use for prediction
final_df = filtered_results_df.merge(filtered_drivers_info, on = 'Abbreviation', how = 'inner' )
final_df[:10]

,Round,Abbreviation,GridPosition,Position,Points,DriverName,TeamId,AvgQualiPosition,AvgRacePosition
0,1,NOR,1.0,1.0,43,Lando Norris,mclaren,2.0,1.5
1,1,VER,3.0,2.0,30,Max Verstappen,red_bull,3.5,3.0
2,1,RUS,4.0,3.0,30,George Russell,mercedes,3.0,3.0
3,1,ANT,16.0,4.0,20,Kimi Antonelli,mercedes,12.0,5.0
4,1,ALB,6.0,5.0,16,Alexander Albon,williams,8.0,6.0
5,1,STR,13.0,6.0,10,Lance Stroll,aston_martin,13.5,7.5
6,1,HUL,17.0,7.0,6,Nico Hülkenberg,sauber,14.5,11.0
7,1,LEC,7.0,8.0,4,Charles Leclerc,ferrari,6.5,13.0
8,1,PIA,2.0,9.0,27,Oscar Piastri,mclaren,1.5,5.0
9,1,HAM,8.0,10.0,1,Lewis Hamilton,ferrari,6.5,14.5


In [50]:
final_df["QualifyingScore"] = (final_df["AvgQualiPosition"] + final_df["GridPosition"]) / 2
final_df

,Round,Abbreviation,GridPosition,Position,Points,DriverName,TeamId,AvgQualiPosition,AvgRacePosition,QualifyingScore
0,1,NOR,1.0,1.0,43,Lando Norris,mclaren,2.0,1.5,1.50
1,1,VER,3.0,2.0,30,Max Verstappen,red_bull,3.5,3.0,3.25
2,1,RUS,4.0,3.0,30,George Russell,mercedes,3.0,3.0,3.50
3,1,ANT,16.0,4.0,20,Kimi Antonelli,mercedes,12.0,5.0,14.00
4,1,ALB,6.0,5.0,16,Alexander Albon,williams,8.0,6.0,7.00
5,1,STR,13.0,6.0,10,Lance Stroll,aston_martin,13.5,7.5,13.25
6,1,HUL,17.0,7.0,6,Nico Hülkenberg,sauber,14.5,11.0,15.75
7,1,LEC,7.0,8.0,4,Charles Leclerc,ferrari,6.5,13.0,6.75
8,1,PIA,2.0,9.0,27,Oscar Piastri,mclaren,1.5,5.0,1.75
9,1,HAM,8.0,10.0,1,Lewis Hamilton,ferrari,6.5,14.5,7.25


In [51]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Round             40 non-null     int64  
 1   Abbreviation      40 non-null     object 
 2   GridPosition      40 non-null     float64
 3   Position          40 non-null     float64
 4   Points            40 non-null     int32  
 5   DriverName        40 non-null     object 
 6   TeamId            40 non-null     object 
 7   AvgQualiPosition  40 non-null     float64
 8   AvgRacePosition   40 non-null     float64
 9   QualifyingScore   40 non-null     float64
dtypes: float64(5), int32(1), int64(1), object(3)
memory usage: 3.1+ KB


In [52]:
# Count and find if there are any null values in the dataset
final_df.isnull().sum()

Round               0
Abbreviation        0
GridPosition        0
Position            0
Points              0
DriverName          0
TeamId              0
AvgQualiPosition    0
AvgRacePosition     0
QualifyingScore     0
dtype: int64

In [54]:
final_df.to_csv('DATA/f1_final_data.csv', index=False)